### Processing of PP sim files to generate states for NWA Presentation
This notebook will use the previously run practically perfect files (on the hard drive) to grab stats to use for visualization.

#### Documentation on creation of each dataset

- April 27 2011 (National)
    - Top-level Recurrence Interval
        - Based off simulations run on PP forecasts back to 1950, used population 'exposed' to tornadoes
    - Magnitude of response
        - Manually assigned given recurrence interval
    - Relative threats to society
        - Tornado
            - For each type of societal impact, a rectangle is plotted for each simulation, showing the simulation's comparison to average
            - Average is based on the national seasonally smoothed average impacts for tornado days
        - Hail/Wind
            - April 27 hail and wind reports were placed on the impacts grids to gather 'expected' impact numbers
            - 'simulation' distributions were generated by sampling 10k times from a normal distribution centerd on the 'expected' number
            - Average is based on the national seasonally smoothed average impacts for hail and wind days
            
    - Median Societal Exposure
        - Tornado
            - For each societal impact type and tornado rating bin, the median value is displayed
            - The color of the grid is tied to the national recurrence interval of that specific rating/impact combo
        - Hail/Wind
            - April 27 hail and wind reports were placed on the impacts grids to gather 'expected' impact numbers for each hail/wind magnitude bin
            - The color of the grid is tied to national recurrence intervals calculated from historic hail/wind reports overlaid on impact grids


- March 25 2021 (FEMA 4)
    - Top-level Recurrence Interval
        - Based off simulations run on PP forecasts back to 1950, used population 'exposed' to tornadoes
    - Magnitude of response
        - Manually assigned given recurrence interval
    - Relative threats to society
        - Tornado
            - For each type of societal impact, a rectangle is plotted for each simulation, showing the simulation's comparison to average
            - Average is based on the FEMA 4 seasonally smoothed average impacts for tornado days
        - Hail/wind
            - March 25 hail and wind reports were placed on the impacts grids to gather 'expected' impact numbers
            - 'simulation' distributions were generated by sampling 10k times from a normal distribution centerd on the 'expected' number
            - Average is based on the FEMA 4 seasonally smoothed average impacts for hail and wind days
            
    - Median Societal Exposure
        - Tornado
            - For each societal impact type and tornado rating bin, the median value is displayed
            - The color of the grid is tied to the FEMA 4 recurrence interval of that specific rating/impact combo
        - Hail/Wind
            - March 25 hail and wind reports were placed on the impacts grids to gather 'expected' impact numbers for each hail/wind magnitude bin
            - The color of the grid is tied to FEMA 4 recurrence intervals calculated from historic hail/wind reports overlaid on impact grids

### Recurrence Interval

#### Data object format

In [79]:
# Recurrence interval
ri = {
    'national' : {
        'all_tors': {
            'people': [],
            'hospitals': [],
            'power plants': [],
            'mobile homes': []
        },
        'sig_tors': {
            'people': [],
            'hospitals': [],
            'power plants': [],
            'mobile homes': []
        },
        'vio_tors': {
            'people': [],
            'hospitals': [],
            'power plants': [],
            'mobile homes': []
        }
    },
    'fema4' : {
        'all_tors': {
            'people': [],
            'hospitals': [],
            'power plants': [],
            'mobile homes': []
        },
        'sig_tors': {
            'people': [],
            'hospitals': [],
            'power plants': [],
            'mobile homes': []
        },
        'vio_tors': {
            'people': [],
            'hospitals': [],
            'power plants': [],
            'mobile homes': []
        }
    }
}

In [1]:
import os
import glob
import random

import pandas as pd
import json

In [56]:
sim_files = glob.glob('/Users/josephpicca/Downloads/simulationFiles/*')

In [68]:
df = pd.read_csv(sim_files[0],sep='|')

In [69]:
df.head()

,sim,population,distance,rating,states,counties,wfos,hospitals,hospitalbeds,mobileparks,mobilehomes,psubstations,plines,slon,slat,elon,elat
0,2,2224,0.5774,1,AL,1069,TAE,0,0,2,50,0,0,-85.3017,31.1810,-85.2968,31.1883
1,3,10864,0.4923,0,AL,1055,BMX,0,0,0,0,2,7,-86.0108,34.0491,-86.0031,34.0522
2,4,496,1.0451,0,AL,1119,BMX,0,0,0,0,0,0,-88.2246,32.5398,-88.2120,32.5505
3,4,544,3.7702,0,AL,1087,BMX,0,0,0,0,0,0,-85.7797,32.4173,-85.7165,32.4287
4,4,4944,0.5589,0,AL,1073,BMX,0,0,1,25,0,0,-86.9067,33.6573,-86.8998,33.6630


In [84]:
# Filter for tornado types
weadata = simdata[simdata['rating'].isin([0,1])]
sigdata = simdata[simdata['rating'].isin([2,3])]
viodata = simdata[simdata['rating'].isin([4,5])]

In [87]:
# For FEMA 4, filter for ['MS','AL','GA','FL','SC','NC','TN','KY']
fema4 = ['MS','AL','GA','FL','SC','NC','TN','KY']
weadata_4 = weadata[weadata['states'].isin(fema4)]
sigdata_4 = sigdata[sigdata['states'].isin(fema4)]
viodata_4 = viodata[viodata['states'].isin(fema4)]

In [98]:
# Collect statistics
vioimpacts_4 = viodata_4.groupby('sim')[['population','hospitals','mobilehomes','psubstations']].sum()
#weaimpacts_4.iloc[0,:].tolist()

Running the actual loop

In [483]:
fema4 = ['MS','AL','GA','FL','SC','NC','TN','KY']

ri_new = {
    'national': {
        'all': [],
        'weak': [],
        'sig': [],
        'vio': []
    },
    'fema4': {
        'all': [],
        'weak': [],
        'sig': [],
        'vio': []
    }
}

for i,file in enumerate(sim_files):
    
    if i % 1000 == 0:
        print(i)
    
    df = pd.read_csv(file,sep='|')
    sim = random.randint(1,10000)
    
    # Filter for that sim 
    simdata = df[df['sim'] == sim]
    
    # National
    # Filter for tornado types
    weadata = simdata[simdata['rating'].isin([0,1])]
    sigdata = simdata[simdata['rating'].isin([2,3])]
    viodata = simdata[simdata['rating'].isin([4,5])]
    
    # Collect statistics
    allimpacts = simdata.groupby('sim')[['population','hospitals','mobilehomes','psubstations']].sum()
    weaimpacts = weadata.groupby('sim')[['population','hospitals','mobilehomes','psubstations']].sum()
    sigimpacts = sigdata.groupby('sim')[['population','hospitals','mobilehomes','psubstations']].sum()
    vioimpacts = viodata.groupby('sim')[['population','hospitals','mobilehomes','psubstations']].sum()
    
    # Try appending weak stats
    try:
        allvals = allimpacts.iloc[0,:].tolist()
        ri_new['national']['all'].append(allvals)
    except:
        pass
    try:
        weavals = weaimpacts.iloc[0,:].tolist()
        ri_new['national']['weak'].append(weavals)
    except:
        pass
    
    # Try appending sig stats
    try:
        sigvals = sigimpacts.iloc[0,:].tolist()
        ri_new['national']['sig'].append(sigvals)
    except:
        pass
    
    # Try appending violent stats
    try:
        viovals = vioimpacts.iloc[0,:].tolist()
        ri_new['national']['vio'].append(viovals)
    except:
        pass
    
    # Fema 
    # Already have the filtered ratings, so we can use those dataframes
    alldata_4 = simdata[simdata['states'].isin(fema4)]
    weadata_4 = weadata[weadata['states'].isin(fema4)]
    sigdata_4 = sigdata[sigdata['states'].isin(fema4)]
    viodata_4 = viodata[viodata['states'].isin(fema4)]
    
    # Collect statistics
    allimpacts_4 = alldata_4.groupby('sim')[['population','hospitals','mobilehomes','psubstations']].sum()
    weaimpacts_4 = weadata_4.groupby('sim')[['population','hospitals','mobilehomes','psubstations']].sum()
    sigimpacts_4 = sigdata_4.groupby('sim')[['population','hospitals','mobilehomes','psubstations']].sum()
    vioimpacts_4 = viodata_4.groupby('sim')[['population','hospitals','mobilehomes','psubstations']].sum()
    
    # Try appending weak stats
    try:
        allvals_4 = allimpacts_4.iloc[0,:].tolist()
        ri_new['fema4']['all'].append(allvals_4)
    except:
        pass
    try:
        weavals_4 = weaimpacts_4.iloc[0,:].tolist()
        ri_new['fema4']['weak'].append(weavals_4)
    except:
        pass
    
    # Try appending sig stats
    try:
        sigvals_4 = sigimpacts_4.iloc[0,:].tolist()
        ri_new['fema4']['sig'].append(sigvals_4)
    except:
        pass
    
    # Try appending violent stats
    try:
        viovals_4 = vioimpacts_4.iloc[0,:].tolist()
        ri_new['fema4']['vio'].append(viovals_4)
    except:
        pass

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000


Saving the ri_new file

In [484]:
with open('./NWA-assets/recur_interval_data.json','w') as fp:
    json.dump(ri_new, fp)

In [27]:
april27 = pd.read_csv('../data/output/examples/20110427/psvs/20110427162927_ns.psv.gz',sep='|')

In [36]:
dec10 = pd.read_csv('./feb22-forecastertraining/output/tripsig.psv.gz',sep='|')

In [160]:
violent = april27[april27['rating'] > 3]
popgroup = violent.groupby('sim').sum()[['population']]
popgroup.reindex(list(range(1,10001)),fill_value=0).describe(percentiles=[0.1,0.5,0.9])

,population
count,10000.000000
mean,96410.977600
std,77017.021621
min,176.000000
10%,23166.400000
50%,77008.000000
90%,192196.800000
max,870240.000000


march 25 2021

In [472]:
march25 = pd.read_csv('../../dev/impacts-app/scripts/impacts-data/output/20210325130600.psv.gz',sep='|')

In [476]:
m25_fema4 = march25[march25['states'].isin(fema4)]

In [477]:
m25_fema4_vio = m25_fema4[m25_fema4['rating'] > 3]

In [493]:
m25_vio_pop = m25_fema4_vio.groupby('sim').sum()[['population']]
m25_vio_pop_med = m25_vio_pop.reindex(list(range(1,10001)),fill_value=0).describe(percentiles=[0.1,0.5,0.9]).loc['50%',:].values[0]

In [485]:
with open('./NWA-assets/recur_interval_data.json') as fp:
    ri_new_tor = json.load(fp)

In [499]:
total = 0

for sim in ri_new_tor['fema4']['vio']:
    if sim[0] > m25_vio_pop_med:
        total += 1
        
f'RI: {70/total} years'

'RI: 1.3461538461538463 years'

### Grid Values (center column)

april 27

In [37]:
dec10_expected = {
    'torn': {
        'data': [{
            'impact': 'population',
            'data': []
        },
        {
            'impact': 'hospitals',
            'data': []
        },
        {
            'impact': 'mobilehomes',
            'data': []
        },
        {
            'impact': 'psubstations',
            'data': []
        }]
    }
}

In [29]:
april27_expected['hail'] = {
        'data': [{
            'impact': 'population',
            'data': []
        },
        {
            'impact': 'hospitals',
            'data': []
        },
        {
            'impact': 'mobilehomes',
            'data': []
        },
        {
            'impact': 'psubstations',
            'data': []
        }]
    }

april27_expected['wind'] = {
        'data': [{
            'impact': 'population',
            'data': []
        },
        {
            'impact': 'hospitals',
            'data': []
        },
        {
            'impact': 'mobilehomes',
            'data': []
        },
        {
            'impact': 'psubstations',
            'data': []
        }]
    }

In [38]:
all1210 = dec10
wea1210 = dec10[dec10['rating'].isin([0,1])]
sig1210 = dec10[dec10['rating'].isin([2,3])]
vio1210 = dec10[dec10['rating'].isin([4,5])]

all1210_quants = all1210.groupby('sim').sum()[['population','hospitals','psubstations','mobilehomes']].describe()
wea1210_quants = wea1210.groupby('sim').sum()[['population','hospitals','psubstations','mobilehomes']].describe()
sig1210_quants = sig1210.groupby('sim').sum()[['population','hospitals','psubstations','mobilehomes']].describe()
vio1210_quants = vio1210.groupby('sim').sum()[['population','hospitals','psubstations','mobilehomes']].describe() #.loc['50%','population']

In [39]:
impacts = ['population','hospitals','mobilehomes','psubstations']

for i, impact in enumerate(impacts):
    
    dec10_expected['torn']['data'][i]['data'].append(all1210_quants.loc['50%',impact])
    dec10_expected['torn']['data'][i]['data'].append(wea1210_quants.loc['50%',impact])
    dec10_expected['torn']['data'][i]['data'].append(sig1210_quants.loc['50%',impact])
    dec10_expected['torn']['data'][i]['data'].append(vio1210_quants.loc['50%',impact])

In [40]:
# april27_expected['torn']['data'][3]['data'][0] = 26.0
# april27_expected['torn']['data'][3]['data'][1] = 22.0
dec10_expected

{'torn': {'data': [{'impact': 'population',
    'data': [57768.0, 38752.0, 9488.0, 2296.0]},
   {'impact': 'hospitals', 'data': [1.0, 1.0, 0.0, 0.0]},
   {'impact': 'mobilehomes', 'data': [325.0, 225.0, 25.0, 0.0]},
   {'impact': 'psubstations', 'data': [14.0, 9.0, 3.0, 1.0]}]}}

In [41]:
with open('./feb22-forecastertraining/postprocessed/20211210_medianvals_ts.json','w') as fp:
    json.dump(dec10_expected, fp)

march 25

In [546]:
march25_expected = {
    'torn': {
        'data': [{
            'impact': 'population',
            'data': []
        },
        {
            'impact': 'hospitals',
            'data': []
        },
        {
            'impact': 'mobilehomes',
            'data': []
        },
        {
            'impact': 'psubstations',
            'data': []
        }]
    }
}

In [615]:
march25_expected['hail'] = {
        'data': [{
            'impact': 'population',
            'data': []
        },
        {
            'impact': 'hospitals',
            'data': []
        },
        {
            'impact': 'mobilehomes',
            'data': []
        },
        {
            'impact': 'psubstations',
            'data': []
        }]
    }

march25_expected['wind'] = {
        'data': [{
            'impact': 'population',
            'data': []
        },
        {
            'impact': 'hospitals',
            'data': []
        },
        {
            'impact': 'mobilehomes',
            'data': []
        },
        {
            'impact': 'psubstations',
            'data': []
        }]
    }

In [548]:
all325 = m25_fema4
wea325 = m25_fema4[m25_fema4['rating'].isin([0,1])]
sig325 = m25_fema4[m25_fema4['rating'].isin([2,3])]
vio325 = m25_fema4[m25_fema4['rating'].isin([4,5])]

all325_quants = all325.groupby('sim').sum()[['population','hospitals','psubstations','mobilehomes']].describe()
wea325_quants = wea325.groupby('sim').sum()[['population','hospitals','psubstations','mobilehomes']].describe()
sig325_quants = sig325.groupby('sim').sum()[['population','hospitals','psubstations','mobilehomes']].describe()
vio325_quants = vio325.groupby('sim').sum()[['population','hospitals','psubstations','mobilehomes']].describe()

In [550]:
impacts = ['population','hospitals','mobilehomes','psubstations']

for i, impact in enumerate(impacts):
    
    march25_expected['torn']['data'][i]['data'].append(all325_quants.loc['50%',impact])
    march25_expected['torn']['data'][i]['data'].append(wea325_quants.loc['50%',impact])
    march25_expected['torn']['data'][i]['data'].append(sig325_quants.loc['50%',impact])
    march25_expected['torn']['data'][i]['data'].append(vio325_quants.loc['50%',impact])

In [551]:
with open('./NWA-assets/20210325_medianvals.json','w') as fp:
    json.dump(march25_expected, fp)

### Daily Climo (left column)

In [12]:
import pandas as pd
import json
import datetime
import numpy as np

In [4]:
date = datetime.datetime.strptime("2021-12-10 00:00:00", "%Y-%m-%d %H:%M:%S")

idx = date.timetuple().tm_yday - 1

In [13]:
with open ('../../dev/impacts-app/data/climo/pop_climo_smAvg.json') as fp:
    pop_climo = json.load(fp)
    
with open ('../../dev/impacts-app/data/climo/hosp_climo_smAvg.json') as fp:
    hosp_climo = json.load(fp)
    
with open ('../../dev/impacts-app/data/climo/pow_climo_smAvg.json') as fp:
    pow_climo = json.load(fp)
    
with open ('../../dev/impacts-app/data/climo/mob_climo_smAvg.json') as fp:
    mob_climo = json.load(fp)

National climate for dec 10

In [14]:
avg_pop = pop_climo['nat']['med'][idx]
avg_hosp = hosp_climo['nat']['med'][idx]
avg_pow = pow_climo['nat']['med'][idx]
avg_mob = mob_climo['nat']['med'][idx]

In [24]:
# df = pd.read_csv('../data/output/examples/20110427/psvs/20110427162927_schools_fema.psv.gz',sep='|')
df = pd.read_csv('./feb22-forecastertraining/output/tripsig.psv.gz',sep='|')

In [25]:
df.head()

,sim,population,distance,rating,states,counties,wfos,hospitals,hospitalbeds,mobileparks,mobilehomes,psubstations,plines,schools,slon,slat,elon,elat
0,1,2272,1.6720,1,AR,5089,LZK,0,0,0,0,0,0,0,-92.6413,36.2793,-92.6188,36.2953
1,1,2800,0.1094,1,IN,18031,IND,0,0,0,0,0,0,0,-85.4381,39.3395,-85.4366,39.3406
2,1,2656,2.4043,1,MO,"29215,29229",SGF,0,0,1,25,0,0,1,-92.2104,37.1401,-92.1718,37.1564
3,1,80,1.3909,0,AR,5141,LZK,0,0,0,0,0,0,0,-92.2127,35.5194,-92.1952,35.5337
4,1,2288,15.4962,4,MO,"29069,29207",PAH,0,0,0,0,0,0,0,-90.0592,36.6059,-89.8333,36.7380


In [26]:
summed = df.groupby('sim').sum()[['population','hospitals','mobilehomes','psubstations']]

pop_rel = np.asarray(summed['population'].to_list()) / avg_pop
hosp_rel = np.asarray(summed['hospitals'].to_list()) / (10*avg_hosp)
mob_rel = np.asarray(summed['mobilehomes'].to_list()) / avg_mob
pow_rel = np.asarray(summed['psubstations'].to_list()) / avg_pow

In [52]:
# Russ Adjustment

pop_rel = np.asarray(summed['population'].to_list()) / avg_pop - 7
hosp_rel = np.asarray(summed['hospitals'].to_list()) / avg_hosp - 5
mob_rel = np.asarray(summed['mobilehomes'].to_list()) / avg_mob - 10
pow_rel = np.asarray(summed['psubstations'].to_list()) / avg_pow - 5

In [27]:
# Save the lists to a json (need to convert to list from np.array to serialize)
# lists will be used in the horizontal bar charts

json_rel = [
    {
        'type':'pop',
         'data':pop_rel.tolist()
    },
    {
        'type':'hosp',
        'data':hosp_rel.tolist()
    },
    {
        'type':'mob',
        'data':mob_rel.tolist()
    },
    {
        'type':'pow',
        'data':pow_rel.tolist()
    }
]

In [28]:
with open('./feb22-forecastertraining/postprocessed/20211210_climorel_ts.json','w') as fp:
    json.dump(json_rel, fp)

march 25 2021

In [539]:
date = datetime.datetime.strptime("2021-03-25 00:00:00", "%Y-%m-%d %H:%M:%S")

idx = date.timetuple().tm_yday - 1

In [540]:
avg_pop = pop_climo['nat']['med'][idx]
avg_hosp = hosp_climo['nat']['med'][idx]
avg_pow = pow_climo['nat']['med'][idx]
avg_mob = mob_climo['nat']['med'][idx]

In [530]:
summed = m25_fema4.groupby('sim').sum()[['population','hospitals','mobilehomes','psubstations']]

pop_rel = np.asarray(summed['population'].to_list()) / avg_pop
hosp_rel = np.asarray(summed['hospitals'].to_list()) / avg_hosp
mob_rel = np.asarray(summed['mobilehomes'].to_list()) / avg_mob
pow_rel = np.asarray(summed['psubstations'].to_list()) / avg_pow

In [532]:
# Save the lists to a json (need to convert to list from np.array to serialize)
# lists will be used in the horizontal bar charts

json_rel = [
    {
        'type':'pop',
         'data':pop_rel.tolist()
    },
    {
        'type':'hosp',
        'data':hosp_rel.tolist()
    },
    {
        'type':'mob',
        'data':mob_rel.tolist()
    },
    {
        'type':'pow',
        'data':pow_rel.tolist()
    }
]

In [533]:
with open('./NWA-assets/20210325_climorel.json','w') as fp:
    json.dump(json_rel, fp)

### Creating hail and wind climo_rel and grid data

Read hail and wind data, grid to get 'median' counts of impacts

In [215]:
from pygridder import pygridder as pgrid

import pyproj

#### Mar 25 2021

In [652]:
hail = pd.read_csv('/Users/josephpicca/Downloads/210325_rpts_hail.csv')
wind = pd.read_csv('/Users/josephpicca/Downloads/210325_rpts_wind.csv')

In [653]:
x1s_h, y1s_h = proj(hail['Lon'].values,hail['Lat'].values)
x1s_w, y1s_w = proj(wind['Lon'].values,wind['Lat'].values)

In [654]:
points_h = G.grid_points(x1s_h,y1s_h)
points_w = G.grid_points(x1s_w,y1s_w)

In [655]:
hail['grid_points'] = points_h
wind['grid_points'] = points_w

In [656]:
p_h = []
h_h = []
pw_h = []
m_h = []

for row in np.arange(hail.shape[0]):
    
    h = hail.iloc[row,:]
    
    p_h.append(pop_grid[h['grid_points']])
    h_h.append(hosp_grid[h['grid_points']])
    pw_h.append(pow_grid[h['grid_points']])
    m_h.append(mob_grid[h['grid_points']])
    
hail['population'] = p_h
hail['hospitals'] = h_h
hail['psubstations'] = pw_h
hail['mobilehomes'] = m_h

In [657]:
hail['Size'] = hail['Size'] / 100

In [625]:
hail.to_csv('./NWA-assets/hail-impacts-3252021.csv',index=False)

In [659]:
p_w = []
h_w = []
pw_w = []
m_w = []

for row in np.arange(wind.shape[0]):
    
    w = wind.iloc[row,:]
    
    p_w.append(pop_grid[w['grid_points']])
    h_w.append(hosp_grid[w['grid_points']])
    pw_w.append(pow_grid[w['grid_points']])
    m_w.append(mob_grid[w['grid_points']])
    
wind['population'] = p_w
wind['hospitals'] = h_w
wind['psubstations'] = pw_w
wind['mobilehomes'] = m_w

wind['Speed'] = wind['Speed'].str.replace('UNK','58').astype('float')

In [660]:
wind['Speed'] = wind['Speed']*0.86897

Make adjustment to 5 random reports to make them sig (to approximately match the sig wind forecast from 3/25/2021)

In [644]:
reports_adj = random.sample(range(0,87), 5)

In [661]:
wind.loc[reports_adj,'Speed'] = 65

In [663]:
wind.to_csv('./NWA-assets/wind-impacts-3252021.csv',index=False)

#### 1955-2019

In [684]:
hail = pd.read_csv('/Users/josephpicca/Downloads/1955-2019_hail.csv')
wind = pd.read_csv('/Users/josephpicca/Downloads/1955-2019_wind.csv')

/Users/josephpicca/opt/anaconda3/envs/impacts-prod-parallel/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [207]:
hail['day'] = pd.to_datetime(hail['date'] + ' ' + hail['time'])
wind['day'] = pd.to_datetime(wind['date'] + ' ' + wind['time'])

In [212]:
ndfd_file = '../../dev/impacts-app/scripts/impacts-data/orig-gis-data/ndfd.npz'
impacts_file = './assets/impact-grids-new.npz'

In [216]:
with np.load(ndfd_file) as NPZ:
    X = NPZ['X']
    Y = NPZ['Y']
    proj = pyproj.Proj(NPZ['srs'].item())

In [217]:
G = pgrid.Gridder(X,Y)

In [222]:
with np.load(impacts_file) as NPZ:
    pop_grid = NPZ['population']
    hosp_grid = NPZ['hospitals']
    pow_grid = NPZ['pstations']
    mob_grid = NPZ['mhomes']

In [276]:
x1s_h, y1s_h = proj(hail['slon'].values,hail['slat'].values)
x1s_w, y1s_w = proj(wind['slon'].values,wind['slat'].values)

In [277]:
points_h = G.grid_points(x1s_h,y1s_h)
points_w = G.grid_points(x1s_w,y1s_w)

In [278]:
hail['grid_points'] = points_h
wind['grid_points'] = points_w

Get impacts data and add to dataframe

In [285]:
p_h = []
h_h = []
pw_h = []
m_h = []

for row in np.arange(hail.shape[0]):
    
    h = hail.iloc[row,:]
    
    p_h.append(pop_grid[h['grid_points']])
    h_h.append(hosp_grid[h['grid_points']])
    pw_h.append(pow_grid[h['grid_points']])
    m_h.append(mob_grid[h['grid_points']])

In [286]:
hail['population'] = p_h
hail['hospitals'] = h_h
hail['psubstations'] = pw_h
hail['mobilehomes'] = m_h

In [288]:
hail.to_csv('./NWA-assets/hail-impacts.csv',index=False)

Do the same for wind

In [289]:
p_w = []
h_w = []
pw_w = []
m_w = []

for row in np.arange(wind.shape[0]):
    
    w = wind.iloc[row,:]
    
    p_w.append(pop_grid[w['grid_points']])
    h_w.append(hosp_grid[w['grid_points']])
    pw_w.append(pow_grid[w['grid_points']])
    m_w.append(mob_grid[w['grid_points']])

In [290]:
wind['population'] = p_w
wind['hospitals'] = h_w
wind['psubstations'] = pw_w
wind['mobilehomes'] = m_w

In [292]:
wind.to_csv('./NWA-assets/wind-impacts.csv',index=False)

#### April 27 National / Hail & Wind Climo Rel

Get the median and SD of national impacts for hail/wind days in april

In [416]:
avg_pop_hail = hail[(hail['yr'] > 2010) & (hail['mo'] == 4)].groupby('date')[['population']].sum().describe().loc['50%',:].values[0]

69712.0

In [423]:
avg_imp_wind = wind[(wind['yr'] > 2010) & (wind['mo'] == 4)].groupby('date')[['population','hospitals','mobilehomes','psubstations']].sum().describe().loc['50%',:].values

avg_pop_wind = avg_imp_wind[0]
avg_hosp_wind = avg_imp_wind[1]
avg_mob_wind = avg_imp_wind[2]
avg_pow_wind = avg_imp_wind[3]

Generate a random distribution of values centered on the impact number from april 27

In [452]:
hail_med = april27_expected['hail']['data'][0]['data'][0]

wind_pop_med = april27_expected['wind']['data'][0]['data'][0]
wind_hosp_med = april27_expected['wind']['data'][1]['data'][0]
wind_mob_med = april27_expected['wind']['data'][2]['data'][0]
wind_pow_med = april27_expected['wind']['data'][3]['data'][0]

In [418]:
hail_dist = np.random.normal(hail_med, 100000, 10000)
hail_dist[hail_dist < 0] = 0

pop_rel_hail = hail_dist / avg_pop_hail

In [465]:
wind_pop_dist = np.rint(np.random.normal(wind_pop_med, 500000, 10000))
wind_pop_dist[wind_pop_dist < 0] = 0

wind_hosp_dist = np.rint(np.random.normal(wind_hosp_med, 30, 10000))
wind_hosp_dist[wind_hosp_dist < 0] = 0

wind_mob_dist = np.rint(np.random.normal(wind_mob_med, 5000, 10000))
wind_mob_dist[wind_mob_dist < 0] = 0

wind_pow_dist = np.rint(np.random.normal(wind_psub_med, 100, 10000))
wind_pow_dist[wind_pow_dist < 0] = 0

pop_rel_wind = wind_pop_dist / avg_pop_wind
hosp_rel_wind = wind_hosp_dist / avg_hosp_wind
mob_rel_wind = wind_mob_dist / avg_mob_wind
pow_rel_wind = wind_pow_dist / avg_pow_wind

In [419]:
json_rel_hail = [
    {
        'type':'pop',
         'data':pop_rel_hail.tolist()
    }
]

In [466]:
json_rel_wind = [
    {
        'type':'pop',
         'data':pop_rel_wind.tolist()
    },
    {
        'type':'hosp',
         'data':hosp_rel_wind.tolist()
    },
    {
        'type':'mob',
         'data':mob_rel_wind.tolist()
    },
    {
        'type':'pow',
         'data':pow_rel_wind.tolist()
    }
]

In [420]:
with open('./NWA-assets/20110427_climorel_hail.json','w') as fp:
    json.dump(json_rel_hail, fp)

In [467]:
with open('./NWA-assets/20110427_climorel_wind.json','w') as fp:
    json.dump(json_rel_wind, fp)

#### March 25 2021 FEMA4 / Hail & Wind Climo Rel

In [598]:
hail_historic = pd.read_csv('./NWA-assets/hail-impacts.csv')

,om,yr,mo,dy,date,time,tz,st,stf,stn,...,f1,f2,f3,f4,day,grid_points,population,hospitals,psubstations,mobilehomes
0,1,1955,1,17,1955-01-17,16:39:00,3,TX,48,1,...,227,0,0,0,1955-01-17 16:39:00,"(211, 421)",32.0,0,0,0
1,2,1955,2,1,1955-02-01,10:30:00,3,MO,29,1,...,135,0,0,0,1955-02-01 10:30:00,"(351, 585)",176.0,0,0,0
2,3,1955,3,3,1955-03-03,13:02:00,3,IL,17,1,...,175,0,0,0,1955-03-03 13:02:00,"(412, 634)",80.0,0,0,0
3,4,1955,3,3,1955-03-03,14:00:00,3,IN,18,1,...,167,0,0,0,1955-03-03 14:00:00,"(370, 676)",176.0,0,1,0
4,5,1955,3,3,1955-03-03,16:00:00,3,IL,17,2,...,31,0,0,0,1955-03-03 16:00:00,"(427, 666)",26160.0,0,6,0


In [601]:
avg_pop_hail_4 = hail_historic[(hail_historic['yr'] > 2010) & (hail_historic['mo'] == 3) & (hail_historic['st'].isin(fema4))].groupby('date')[['population']].sum().describe().loc['50%',:].values[0]

In [602]:
wind_historic = pd.read_csv('./NWA-assets/wind-impacts.csv')

/Users/josephpicca/opt/anaconda3/envs/impacts-prod-parallel/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [603]:
avg_imp_wind_4 = wind_historic[(wind_historic['yr'] > 2010) & (wind_historic['mo'] == 3) & (wind_historic['st'].isin(fema4))].groupby('date')[['population','hospitals','mobilehomes','psubstations']].sum().describe().loc['50%',:].values

avg_pop_wind_4 = avg_imp_wind_4[0]
avg_hosp_wind_4 = avg_imp_wind_4[1]
avg_mob_wind_4 = avg_imp_wind_4[2]
avg_pow_wind_4 = avg_imp_wind_4[3]

Get 'median' val data for hail, wind. In reality, this is just the calculated impact numbers

In [669]:
hail_med = march25_expected['hail']['data'][0]['data'][0]

wind_pop_med = march25_expected['wind']['data'][0]['data'][0]
wind_hosp_med = march25_expected['wind']['data'][1]['data'][0]
wind_mob_med = march25_expected['wind']['data'][2]['data'][0]
wind_pow_med = march25_expected['wind']['data'][3]['data'][0]

In [670]:
hail_dist = np.random.normal(hail_med, 100000, 10000)
hail_dist[hail_dist < 0] = 0

pop_rel_hail = hail_dist / avg_pop_hail_4

In [672]:
wind_pop_dist = np.rint(np.random.normal(wind_pop_med, 500000, 10000))
wind_pop_dist[wind_pop_dist < 0] = 0

wind_hosp_dist = np.rint(np.random.normal(wind_hosp_med, 30, 10000))
wind_hosp_dist[wind_hosp_dist < 0] = 0

wind_mob_dist = np.rint(np.random.normal(wind_mob_med, 5000, 10000))
wind_mob_dist[wind_mob_dist < 0] = 0

wind_pow_dist = np.rint(np.random.normal(wind_psub_med, 100, 10000))
wind_pow_dist[wind_pow_dist < 0] = 0

pop_rel_wind = wind_pop_dist / avg_pop_wind_4
hosp_rel_wind = wind_hosp_dist / avg_hosp_wind_4
mob_rel_wind = wind_mob_dist / avg_mob_wind_4
pow_rel_wind = wind_pow_dist / avg_pow_wind_4

In [675]:
json_rel_hail = [
    {
        'type':'pop',
         'data':pop_rel_hail.tolist()
    }
]

In [676]:
json_rel_wind = [
    {
        'type':'pop',
         'data':pop_rel_wind.tolist()
    },
    {
        'type':'hosp',
         'data':hosp_rel_wind.tolist()
    },
    {
        'type':'mob',
         'data':mob_rel_wind.tolist()
    },
    {
        'type':'pow',
         'data':pow_rel_wind.tolist()
    }
]

In [677]:
with open('./NWA-assets/20210325_climorel_hail.json','w') as fp:
    json.dump(json_rel_hail, fp)

In [678]:
with open('./NWA-assets/20210325_climorel_wind.json','w') as fp:
    json.dump(json_rel_wind, fp)

#### March 25 FEMA 4 Hail/Wind Grid

In [664]:
h25_a = hail[(hail['Size'] > 0.99)]
h25_1 = hail[(hail['Size'] > 0.99) & (hail['Size'] < 1.99)]
h25_2 = hail[(hail['Size'] > 1.99) & (hail['Size'] < 3.99)]
h25_4 = hail[(hail['Size'] > 3.99)]

w25_a = wind[(wind['Speed'] > 49.9)]
w25_50 = wind[(wind['Speed'] > 49.9) & (wind['Speed'] < 64.9)]
w25_65 = wind[(wind['Speed'] > 64.9) & (wind['Speed'] < 79.9)]
w25_80 = wind[(wind['Speed'] > 79.9)]

In [666]:
impacts = ['population','hospitals','mobilehomes','psubstations']

for i,entry in enumerate(march25_expected['hail']['data']):
    
    arr = [float(h25_a[impacts[i]].sum()), float(h25_1[impacts[i]].sum()), float(h25_2[impacts[i]].sum()), float(h25_4[impacts[i]].sum())]
    
    entry['data'] = arr
    
for i,entry in enumerate(march25_expected['wind']['data']):
    
    arr = [float(w25_a[impacts[i]].sum()), float(w25_50[impacts[i]].sum()), float(w25_65[impacts[i]].sum()), float(w25_80[impacts[i]].sum())]
    
    entry['data'] = arr

In [667]:
march25_expected

{'torn': {'data': [{'impact': 'population',
    'data': [59512.0, 37720.0, 12144.0, 2784.0]},
   {'impact': 'hospitals', 'data': [1.0, 1.0, 0.0, 0.0]},
   {'impact': 'mobilehomes', 'data': [325.0, 200.0, 50.0, 0.0]},
   {'impact': 'psubstations', 'data': [20.0, 13.0, 4.0, 1.0]}]},
 'hail': {'data': [{'impact': 'population',
    'data': [132352.0, 130752.0, 1600.0, 0.0]},
   {'impact': 'hospitals', 'data': [7.0, 7.0, 0.0, 0.0]},
   {'impact': 'mobilehomes', 'data': [1100.0, 1100.0, 0.0, 0.0]},
   {'impact': 'psubstations', 'data': [51.0, 51.0, 0.0, 0.0]}]},
 'wind': {'data': [{'impact': 'population',
    'data': [469728.0, 409872.0, 59856.0, 0.0]},
   {'impact': 'hospitals', 'data': [4.0, 4.0, 0.0, 0.0]},
   {'impact': 'mobilehomes', 'data': [2225.0, 2200.0, 25.0, 0.0]},
   {'impact': 'psubstations', 'data': [141.0, 125.0, 16.0, 0.0]}]}}

In [668]:
with open('./NWA-assets/20210325_medianvals.json','w') as fp:
    json.dump(march25_expected, fp)

#### April 27 National Hail/Wind Grid

In [293]:
# Get just april 27 2011
bdt = datetime.datetime(2011,4,27,12,0)
edt = datetime.datetime(2011,4,28,12,0)

h27_a = hail[(hail['day'] > bdt) & (hail['day'] < edt) & (hail['mag'] > 0.99)]
h27_1 = hail[(hail['day'] > bdt) & (hail['day'] < edt) & (hail['mag'] > 0.99) & (hail['mag'] < 1.99)]
h27_2 = hail[(hail['day'] > bdt) & (hail['day'] < edt) & (hail['mag'] > 1.99) & (hail['mag'] < 3.99)]
h27_4 = hail[(hail['day'] > bdt) & (hail['day'] < edt) & (hail['mag'] > 3.99)]

w27_a = wind[(wind['day'] > bdt) & (wind['day'] < edt) & (wind['mag'] > 49.9)]
w27_50 = wind[(wind['day'] > bdt) & (wind['day'] < edt) & (wind['mag'] > 49.9) & (wind['mag'] < 64.9)]
w27_65 = wind[(wind['day'] > bdt) & (wind['day'] < edt) & (wind['mag'] > 64.9) & (wind['mag'] < 79.9)]
w27_80 = wind[(wind['day'] > bdt) & (wind['day'] < edt) & (wind['mag'] > 79.9)]

In [315]:
impacts = ['population','hospitals','mobilehomes','psubstations']

for i,entry in enumerate(april27_expected['hail']['data']):
    
    arr = [float(h27_a[impacts[i]].sum()), float(h27_1[impacts[i]].sum()), float(h27_2[impacts[i]].sum()), float(h27_4[impacts[i]].sum())]
    
    entry['data'] = arr
    
for i,entry in enumerate(april27_expected['wind']['data']):
    
    arr = [float(w27_a[impacts[i]].sum()), float(w27_50[impacts[i]].sum()), float(w27_65[impacts[i]].sum()), float(w27_80[impacts[i]].sum())]
    
    entry['data'] = arr

In [316]:
april27_expected

{'torn': {'data': [{'impact': 'population',
    'data': [380080.0, 168176.0, 116112.0, 77008.0]},
   {'impact': 'hospitals', 'data': [8.0, 3.0, 2.0, 1.0]},
   {'impact': 'mobilehomes', 'data': [2275.0, 950.0, 700.0, 475.0]},
   {'impact': 'psubstations', 'data': [96.0, 41.0, 32.0, 22.0]}]},
 'hail': {'data': [{'impact': 'population',
    'data': [298576.0, 261808.0, 36064.0, 704.0]},
   {'impact': 'hospitals', 'data': [51.0, 45.0, 6.0, 0.0]},
   {'impact': 'mobilehomes', 'data': [5350.0, 5100.0, 200.0, 50.0]},
   {'impact': 'psubstations', 'data': [204.0, 169.0, 30.0, 5.0]}]},
 'wind': {'data': [{'impact': 'population',
    'data': [693264.0, 655616.0, 35136.0, 2512.0]},
   {'impact': 'hospitals', 'data': [51.0, 50.0, 1.0, 0.0]},
   {'impact': 'mobilehomes', 'data': [7275.0, 6700.0, 550.0, 25.0]},
   {'impact': 'psubstations', 'data': [274.0, 264.0, 10.0, 0.0]}]}}

In [317]:
with open('./NWA-assets/20110427_medianvals.json','w') as fp:
    json.dump(april27_expected, fp)

recurrence interval for hail,wind

In [359]:
ri_new = {
    'hail': {
        'national': {
            'all': [],
            'weak': [],
            'sig': [],
            'vio': []
        },
        'fema4': {
            'all': [],
            'weak': [],
            'sig': [],
            'vio': []
        }
    },
    'wind': {
        'national': {
            'all': [],
            'weak': [],
            'sig': [],
            'vio': []
        },
        'fema4': {
            'all': [],
            'weak': [],
            'sig': [],
            'vio': []
        }
    }
}

In [339]:
# Create date array
my_hour = datetime.datetime.strptime('1200', '%H%M').hour
my_minute = datetime.datetime.strptime('1200', '%H%M').minute

bdt = datetime.datetime(1955,1,1,my_hour,my_minute)
edt = datetime.datetime(2019,12,31,my_hour,my_minute)

# Create list index of datetimes with a frequency of one per day
dts = pd.date_range(bdt, edt, freq='D')
bdts, edts = dts[:-1],dts[1:]

In [360]:
for bdt,edt in zip(bdts[:],edts[:]):
    
    _wind = wind[(wind['day'] > bdt) & (wind['day'] < edt)]
    _hail = hail[(hail['day'] > bdt) & (hail['day'] < edt)]
    
    _wind_4 = _wind[_wind['st'].isin(fema4)]
    _hail_4 = _hail[_hail['st'].isin(fema4)]

    
    
    # National
    # Filter for types
    h27_a = _hail[(_hail['day'] > bdt) & (_hail['day'] < edt) & (_hail['mag'] > 0.99)][['population','hospitals','mobilehomes','psubstations']].sum()
    h27_1 = _hail[(_hail['day'] > bdt) & (_hail['day'] < edt) & (_hail['mag'] > 0.99) & (_hail['mag'] < 1.99)][['population','hospitals','mobilehomes','psubstations']].sum()
    h27_2 = _hail[(_hail['day'] > bdt) & (_hail['day'] < edt) & (_hail['mag'] > 1.99) & (_hail['mag'] < 3.99)][['population','hospitals','mobilehomes','psubstations']].sum()
    h27_4 = _hail[(_hail['day'] > bdt) & (_hail['day'] < edt) & (_hail['mag'] > 3.99)][['population','hospitals','mobilehomes','psubstations']].sum()

    w27_a = _wind[(_wind['day'] > bdt) & (_wind['day'] < edt) & (_wind['mag'] > 49.9)][['population','hospitals','mobilehomes','psubstations']].sum()
    w27_50 = _wind[(_wind['day'] > bdt) & (_wind['day'] < edt) & (_wind['mag'] > 49.9) & (_wind['mag'] < 64.9)][['population','hospitals','mobilehomes','psubstations']].sum()
    w27_65 = _wind[(_wind['day'] > bdt) & (_wind['day'] < edt) & (_wind['mag'] > 64.9) & (_wind['mag'] < 79.9)][['population','hospitals','mobilehomes','psubstations']].sum()
    w27_80 = _wind[(_wind['day'] > bdt) & (_wind['day'] < edt) & (_wind['mag'] > 79.9)][['population','hospitals','mobilehomes','psubstations']].sum()
    
    
    # fema 4 filtering
    h27_a_4 = _hail_4[(_hail_4['day'] > bdt) & (_hail_4['day'] < edt) & (_hail_4['mag'] > 0.99)][['population','hospitals','mobilehomes','psubstations']].sum()
    h27_1_4 = _hail_4[(_hail_4['day'] > bdt) & (_hail_4['day'] < edt) & (_hail_4['mag'] > 0.99) & (_hail_4['mag'] < 1.99)][['population','hospitals','mobilehomes','psubstations']].sum()
    h27_2_4 = _hail_4[(_hail_4['day'] > bdt) & (_hail_4['day'] < edt) & (_hail_4['mag'] > 1.99) & (_hail_4['mag'] < 3.99)][['population','hospitals','mobilehomes','psubstations']].sum()
    h27_4_4 = _hail_4[(_hail_4['day'] > bdt) & (_hail_4['day'] < edt) & (_hail_4['mag'] > 3.99)][['population','hospitals','mobilehomes','psubstations']].sum()

    w27_a_4 = _wind_4[(_wind_4['day'] > bdt) & (_wind_4['day'] < edt) & (_wind_4['mag'] > 49.9)][['population','hospitals','mobilehomes','psubstations']].sum()
    w27_50_4 = _wind_4[(_wind_4['day'] > bdt) & (_wind_4['day'] < edt) & (_wind_4['mag'] > 49.9) & (_wind_4['mag'] < 64.9)][['population','hospitals','mobilehomes','psubstations']].sum()
    w27_65_4 = _wind_4[(_wind_4['day'] > bdt) & (_wind_4['day'] < edt) & (_wind_4['mag'] > 64.9) & (_wind_4['mag'] < 79.9)][['population','hospitals','mobilehomes','psubstations']].sum()
    w27_80_4 = _wind_4[(_wind_4['day'] > bdt) & (_wind_4['day'] < edt) & (_wind_4['mag'] > 79.9)][['population','hospitals','mobilehomes','psubstations']].sum()
    
    
    
    
    
    # Hail
    # Try appending weak stats
    allvals = h27_a.values    
    if allvals.sum() != 0:
        ri_new['hail']['national']['all'].append(list(allvals))

    weavals = h27_1.values    
    if weavals.sum() != 0:
        ri_new['hail']['national']['weak'].append(list(weavals))

    
    # Try appending sig stats
    sigvals = h27_2.values
    if sigvals.sum() != 0:
        ri_new['hail']['national']['sig'].append(list(sigvals))

    # Try appending violent stats
    viovals = h27_4.values
    if viovals.sum() != 0:
        ri_new['hail']['national']['vio'].append(list(viovals))

    
    
    # Wind
    # Try appending weak stats
    allvals = w27_a.values
    if allvals.sum() != 0:
        ri_new['wind']['national']['all'].append(list(allvals))

    weavals = w27_50.values
    if weavals.sum() != 0:
        ri_new['wind']['national']['weak'].append(list(weavals))

    
    # Try appending sig stats
    sigvals = w27_65.values
    if sigvals.sum() != 0:
        ri_new['wind']['national']['sig'].append(list(sigvals))

    
    # Try appending violent stats
    viovals = w27_80.values
    if viovals.sum() != 0:
        ri_new['wind']['national']['vio'].append(list(viovals))
    
    
    # fema4 appending
    
    # Hail
    # Try appending weak stats
    
    allvals = h27_a_4.values
    if allvals.sum() != 0:
        ri_new['hail']['fema4']['all'].append(list(allvals))

    
    weavals = h27_1_4.values
    if weavals.sum() != 0:
        ri_new['hail']['fema4']['weak'].append(list(weavals))
    
    
    # Try appending sig stats
    sigvals = h27_2_4.values
    if sigvals.sum() != 0:
        ri_new['hail']['fema4']['sig'].append(list(sigvals))

    
    # Try appending violent stats
    viovals = h27_4_4.values
    if viovals.sum() != 0:
        ri_new['hail']['fema4']['vio'].append(list(viovals))

    
    
    # Wind
    # Try appending weak stats
    
    allvals = w27_a_4.values
    if allvals.sum() != 0:
        ri_new['wind']['fema4']['all'].append(list(allvals))

    
    weavals = w27_50_4.values
    if weavals.sum() != 0:
        ri_new['wind']['fema4']['weak'].append(list(weavals))

    
    # Try appending sig stats
    sigvals = w27_65_4.values
    if sigvals.sum() != 0:
        ri_new['wind']['fema4']['sig'].append(list(sigvals))
        
    
    # Try appending violent stats
    
    viovals = w27_80_4.values
    if viovals.sum() != 0:
        ri_new['wind']['fema4']['vio'].append(list(viovals))



In [368]:
with open('./NWA-assets/recur_interval_data_hw.json','w') as fp:
    json.dump(ri_new, fp)